## Rafi Fadhlillah
## 121450143

### menggunakan regularisasi L1

In [1]:
import numpy as np
import pandas as pd

# Fungsi aktivasi Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Fungsi update Adam optimizer
def adam_update(weight, gradient, m, v, t, learning_rate, beta1, beta2, epsilon=1e-8):
    m = beta1 * m + (1 - beta1) * gradient
    v = beta2 * v + (1 - beta2) * (gradient ** 2)
    m_hat = m / (1 - beta1 ** t)
    v_hat = v / (1 - beta2 ** t)
    weight += learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
    return weight, m, v

# Neural network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, l1_lambda=0.0, l2_lambda=0.0):
        # Inisialisasi bobot dan bias
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_hidden = np.random.randn(hidden_size)
        self.bias_output = np.random.randn(output_size)

        # Regularisasi
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

        # Adam optimizer parameters
        self.m_ih = np.zeros_like(self.weights_input_hidden)
        self.v_ih = np.zeros_like(self.weights_input_hidden)
        self.m_ho = np.zeros_like(self.weights_hidden_output)
        self.v_ho = np.zeros_like(self.weights_hidden_output)
        self.beta1, self.beta2 = 0.9, 0.999
        self.t = 1

    def forward(self, inputs):
        # Propagasi maju untuk batch
        self.hidden_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = sigmoid(self.output_input)
        return self.predicted_output

    def compute_loss(self, targets):
        # Hitung MSE loss
        error = targets - self.predicted_output
        mse_loss = np.mean(np.square(error))

        # regularisasi L1 dan L2
        l1_penalty = self.l1_lambda * (np.sum(np.abs(self.weights_input_hidden)) + np.sum(np.abs(self.weights_hidden_output)))
        l2_penalty = self.l2_lambda * (np.sum(np.square(self.weights_input_hidden)) + np.sum(np.square(self.weights_hidden_output)))

        # hitung total loss menggunakan l1
        total_loss = mse_loss + l1_penalty
        return total_loss

    def backward(self, inputs, targets, learning_rate):
        # Backpropagation untuk batch
        error = targets - self.predicted_output
        delta_output = error * sigmoid_derivative(self.predicted_output)
        error_hidden = delta_output.dot(self.weights_hidden_output.T)
        delta_hidden = error_hidden * sigmoid_derivative(self.hidden_output)

        # Update bobot menggunakan Adam optimizer
        self.weights_hidden_output, self.m_ho, self.v_ho = adam_update(
            self.weights_hidden_output, np.dot(self.hidden_output.T, delta_output),
            self.m_ho, self.v_ho, self.t, learning_rate, self.beta1, self.beta2
        )
        self.weights_input_hidden, self.m_ih, self.v_ih = adam_update(
            self.weights_input_hidden, np.dot(inputs.T, delta_hidden),
            self.m_ih, self.v_ih, self.t, learning_rate, self.beta1, self.beta2
        )

        self.t += 1

    def train(self, training_data, targets, epochs, learning_rate, batch_size):
        error_history = []  # Untuk menyimpan error di setiap epoch
        for epoch in range(epochs):
            total_error = 0
            for i in range(0, len(training_data), batch_size):
                # Buat batch
                batch_inputs = training_data[i:i + batch_size]
                batch_targets = targets[i:i + batch_size].reshape(-1, 1)

                # Forward pass
                self.forward(batch_inputs)

                # Backward pass
                self.backward(batch_inputs, batch_targets, learning_rate)

                # Hitung error dan tambahkan regularisasi
                total_error += self.compute_loss(batch_targets)

            error_history.append(total_error / (len(training_data) / batch_size))

            # Print error setiap 10 epoch
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Error: {total_error / (len(training_data) / batch_size)}")

    def predict(self, inputs):
        return self.forward(inputs)

# Load dataset
url = '/content/heart.csv'
data = pd.read_csv(url)
training_data = data.iloc[:, :-1].values
targets = data.iloc[:, -1].values

# Parameter jaringan saraf
input_size = training_data.shape[1]
hidden_size = 5  # Jumlah node di hidden layer
output_size = 2  # Jumlah output
learning_rate = 0.01
epochs = 400
batch_size = 10

# Inisialisasi dan latih jaringan saraf dengan regularisasi L1 atau L2
nn = NeuralNetwork(input_size, hidden_size, output_size,
                   l1_lambda=0.01)
nn.train(training_data, targets, epochs, learning_rate, batch_size)

# Test network (Optional)
# for inputs in training_data:
#     prediction = nn.predict(inputs.reshape(1, -1))  # Ubah input ke bentuk batch
#     print(f"Predicted Output: {prediction}")


<ipython-input-1-0253f3254ec1>:6: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Error: 1.017731361625325
Epoch 10, Error: 0.9346722759708983
Epoch 20, Error: 0.933389854148848
Epoch 30, Error: 0.9313080648531002
Epoch 40, Error: 0.9287947620145683
Epoch 50, Error: 0.9262766555773032
Epoch 60, Error: 0.929302805872994
Epoch 70, Error: 0.9341647985733299
Epoch 80, Error: 0.9384912740875743
Epoch 90, Error: 0.9423549448847839
Epoch 100, Error: 0.9458398694085249
Epoch 110, Error: 0.9490193590170156
Epoch 120, Error: 0.9519522761557763
Epoch 130, Error: 0.9546847511823361
Epoch 140, Error: 0.9572528421659363
Epoch 150, Error: 0.9596848987186871
Epoch 160, Error: 0.9620034103959373
Epoch 170, Error: 0.9642263943438264
Epoch 180, Error: 0.9663684260214526
Epoch 190, Error: 0.9684414061107389
Epoch 200, Error: 0.9704551351835998
Epoch 210, Error: 0.9724177483547443
Epoch 220, Error: 0.9743360473815547
Epoch 230, Error: 0.9762157570100642
Epoch 240, Error: 0.9780617248188714
Epoch 250, Error: 0.979878078494837
Epoch 260, Error: 0.9816683507167423
Epoch 270, Error

### Tanpa L1

In [2]:
import numpy as np
import pandas as pd

# Fungsi aktivasi Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Fungsi update Adam optimizer
def adam_update(weight, gradient, m, v, t, learning_rate, beta1, beta2, epsilon=1e-8):
    m = beta1 * m + (1 - beta1) * gradient
    v = beta2 * v + (1 - beta2) * (gradient ** 2)
    m_hat = m / (1 - beta1 ** t)
    v_hat = v / (1 - beta2 ** t)
    weight += learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
    return weight, m, v

# Neural network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, l1_lambda=0.0, l2_lambda=0.0):
        # Inisialisasi bobot dan bias
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_hidden = np.random.randn(hidden_size)
        self.bias_output = np.random.randn(output_size)

        # Regularisasi
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

        # Adam optimizer parameters
        self.m_ih = np.zeros_like(self.weights_input_hidden)
        self.v_ih = np.zeros_like(self.weights_input_hidden)
        self.m_ho = np.zeros_like(self.weights_hidden_output)
        self.v_ho = np.zeros_like(self.weights_hidden_output)
        self.beta1, self.beta2 = 0.9, 0.999
        self.t = 1

    def forward(self, inputs):
        # Propagasi maju untuk batch
        self.hidden_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = sigmoid(self.output_input)
        return self.predicted_output

    def compute_loss(self, targets):
        # Hitung MSE loss
        error = targets - self.predicted_output
        mse_loss = np.mean(np.square(error))

        # regularisasi L1 dan L2
        l1_penalty = self.l1_lambda * (np.sum(np.abs(self.weights_input_hidden)) + np.sum(np.abs(self.weights_hidden_output)))
        l2_penalty = self.l2_lambda * (np.sum(np.square(self.weights_input_hidden)) + np.sum(np.square(self.weights_hidden_output)))

        # hitung total loss tanpa menggunakan l1
        total_loss = mse_loss
        return total_loss

    def backward(self, inputs, targets, learning_rate):
        # Backpropagation untuk batch
        error = targets - self.predicted_output
        delta_output = error * sigmoid_derivative(self.predicted_output)
        error_hidden = delta_output.dot(self.weights_hidden_output.T)
        delta_hidden = error_hidden * sigmoid_derivative(self.hidden_output)

        # Update bobot menggunakan Adam optimizer
        self.weights_hidden_output, self.m_ho, self.v_ho = adam_update(
            self.weights_hidden_output, np.dot(self.hidden_output.T, delta_output),
            self.m_ho, self.v_ho, self.t, learning_rate, self.beta1, self.beta2
        )
        self.weights_input_hidden, self.m_ih, self.v_ih = adam_update(
            self.weights_input_hidden, np.dot(inputs.T, delta_hidden),
            self.m_ih, self.v_ih, self.t, learning_rate, self.beta1, self.beta2
        )

        self.t += 1

    def train(self, training_data, targets, epochs, learning_rate, batch_size):
        error_history = []  # Untuk menyimpan error di setiap epoch
        for epoch in range(epochs):
            total_error = 0
            for i in range(0, len(training_data), batch_size):
                # Buat batch
                batch_inputs = training_data[i:i + batch_size]
                batch_targets = targets[i:i + batch_size].reshape(-1, 1)

                # Forward pass
                self.forward(batch_inputs)

                # Backward pass
                self.backward(batch_inputs, batch_targets, learning_rate)

                # Hitung error dan tambahkan regularisasi
                total_error += self.compute_loss(batch_targets)

            error_history.append(total_error / (len(training_data) / batch_size))

            # Print error setiap 10 epoch
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Error: {total_error / (len(training_data) / batch_size)}")

    def predict(self, inputs):
        return self.forward(inputs)

# Load dataset
url = '/content/heart.csv'
data = pd.read_csv(url)
training_data = data.iloc[:, :-1].values
targets = data.iloc[:, -1].values

# Parameter jaringan saraf
input_size = training_data.shape[1]
hidden_size = 5  # Jumlah node di hidden layer
output_size = 2  # Jumlah output
learning_rate = 0.01
epochs = 400
batch_size = 10

# Inisialisasi dan latih jaringan saraf dengan regularisasi L1 atau L2
nn = NeuralNetwork(input_size, hidden_size, output_size,
                   l1_lambda=0.01)
nn.train(training_data, targets, epochs, learning_rate, batch_size)

# Test network (Optional)
# for inputs in training_data:
#     prediction = nn.predict(inputs.reshape(1, -1))  # Ubah input ke bentuk batch
#     print(f"Predicted Output: {prediction}")


<ipython-input-2-8644debebc0d>:6: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Error: 0.3025428763135673
Epoch 10, Error: 0.2602609341717487
Epoch 20, Error: 0.2588246381994896
Epoch 30, Error: 0.2586659029076323
Epoch 40, Error: 0.2585942018447961
Epoch 50, Error: 0.25854548617741335
Epoch 60, Error: 0.25851059978714036
Epoch 70, Error: 0.25848522665671847
Epoch 80, Error: 0.25846652987421004
Epoch 90, Error: 0.25845256370494607
Epoch 100, Error: 0.25844198696012316
Epoch 110, Error: 0.25843387050257977
Epoch 120, Error: 0.2584275648468784
Epoch 130, Error: 0.2584226105897536
Epoch 140, Error: 0.2584186784427682
Epoch 150, Error: 0.2584155291241267
Epoch 160, Error: 0.25841298638457566
Epoch 170, Error: 0.2584109186964006
Epoch 180, Error: 0.25840922668163335
Epoch 190, Error: 0.25840783437444576
Epoch 200, Error: 0.2584066830720618
Epoch 210, Error: 0.2584057269533983
Epoch 220, Error: 0.2584049299192595
Epoch 230, Error: 0.2584042632866791
Epoch 240, Error: 0.25840370408745644
Epoch 250, Error: 0.2584032337989162
Epoch 260, Error: 0.2584028373872554
E

In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
url = '/content/heart.csv'
data = pd.read_csv(url)
data.head(3)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1


In [21]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state=50)
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (196, 13)
y_train shape:  (196,)
X_test shape:  (107, 13)
y_test shape:  (107,)


## Dropout 0,2

In [22]:
model = Sequential()
model.add(Dense(5, input_shape=(13,), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.compile(optimizer='adam', loss='mse',
              metrics=['accuracy'])

In [25]:
model.fit(x=X_train, y=y_train, epochs = 300)

Epoch 1/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5845 - loss: 0.2614
Epoch 2/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5145 - loss: 0.2798 
Epoch 3/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5105 - loss: 0.2829 
Epoch 4/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5276 - loss: 0.2745 
Epoch 5/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5420 - loss: 0.2562 
Epoch 6/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5404 - loss: 0.2528 
Epoch 7/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5578 - loss: 0.2600 
Epoch 8/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4932 - loss: 0.2898 
Epoch 9/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5276 - loss: 0.2694 
Epoch 10/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6043 - loss: 0.2527 
Epoch 11/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5201 - loss: 0.2667 
Epoch 12/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5217 - loss

In [26]:
model.evaluate(X_test, y_test, verbose=0)

[0.24699491262435913, 0.5794392228126526]

In [27]:
model.evaluate(X_train, y_train, verbose=0)

[0.24936191737651825, 0.5255101919174194]

## Dropout 0,3

In [11]:
model = Sequential()
model.add(Dense(5, input_shape=(13,), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam', loss='mse',
              metrics=['accuracy'])

In [13]:
model.fit(x=X_train, y=y_train, epochs = 300)

Epoch 1/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4833 - loss: 0.2806
Epoch 2/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4777 - loss: 0.2934  
Epoch 3/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5539 - loss: 0.2612 
Epoch 4/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5357 - loss: 0.2714  
Epoch 5/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4854 - loss: 0.2855 
Epoch 6/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4913 - loss: 0.2623  
Epoch 7/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5419 - loss: 0.2577  
Epoch 8/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4885 - loss: 0.3004 
Epoch 9/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5699 - loss: 0.2610 
Epoch 10/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5268 - loss: 0.2760 
Epoch 11/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5088 - loss: 0.2736 
Epoch 12/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5355 - 

In [14]:
model.evaluate(X_test, y_test, verbose=0)

[0.2345503717660904, 0.6315789222717285]

In [15]:
model.evaluate(X_train, y_train, verbose=0)

[0.23421266674995422, 0.6431717872619629]

### Dropout 0,5

In [16]:
model = Sequential()
model.add(Dense(5, input_shape=(13,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.compile(optimizer='adam', loss='mse',
              metrics=['accuracy'])

In [18]:
model.fit(x=X_train, y=y_train, epochs = 300)

Epoch 1/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4717 - loss: 0.2907
Epoch 2/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4786 - loss: 0.3117 
Epoch 3/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4929 - loss: 0.2940 
Epoch 4/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4954 - loss: 0.2930 
Epoch 5/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5428 - loss: 0.2823 
Epoch 6/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5825 - loss: 0.2832 
Epoch 7/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5312 - loss: 0.2834 
Epoch 8/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5222 - loss: 0.2970 
Epoch 9/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4999 - loss: 0.2914 
Epoch 10/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5361 - loss: 0.2761 
Epoch 11/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5385 - loss: 0.2857 
Epoch 12/300
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5222 - loss

In [19]:
model.evaluate(X_test, y_test, verbose=0)

[0.24835604429244995, 0.5657894611358643]

In [20]:
model.evaluate(X_train, y_train, verbose=0)

[0.24914850294589996, 0.5374449491500854]